# 1 Colors_1 classification model

In [148]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

### Define image and batch size

In [149]:
IMAGE_SIZE = 90
BATCH_SIZE = 32

### Import train, test, and val

In [150]:
colors_1_train = utils.image_dataset_from_directory("datasets/data/split/colors_1/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_1_val = utils.image_dataset_from_directory("datasets/data/split/colors_1/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_1_test = utils.image_dataset_from_directory("datasets/data/split/colors_1/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 7267 files belonging to 15 classes.
Found 1550 files belonging to 15 classes.
Found 1556 files belonging to 15 classes.


### Augmentation

In [151]:
colors_1_data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

### Define Model

In [152]:
colors_1_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

x = colors_1_data_augmentation(colors_1_inputs)

x = layers.Rescaling(1./255)(colors_1_inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.GlobalAveragePooling2D()(x)

x = layers.Flatten()(x)

colors_1_outputs = layers.Dense(15, activation="softmax")(x)

colors_1_model = keras.Model(inputs=colors_1_inputs, outputs=colors_1_outputs)

colors_1_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

colors_1_model.summary()

Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 90, 90, 3)]       0         
                                                                 
 rescaling_26 (Rescaling)    (None, 90, 90, 3)         0         
                                                                 
 conv2d_115 (Conv2D)         (None, 88, 88, 32)        896       
                                                                 
 max_pooling2d_89 (MaxPoolin  (None, 44, 44, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_116 (Conv2D)         (None, 42, 42, 64)        18496     
                                                                 
 max_pooling2d_90 (MaxPoolin  (None, 21, 21, 64)       0         
 g2D)                                                     

In [153]:
colors_1_callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/colors_1_3_secondAug_checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

### Run Model

In [154]:
history = colors_1_model.fit(
  colors_1_train.cache(),
  epochs=30,
  callbacks=colors_1_callbacks_list,
  validation_data=colors_1_val.cache()
)

Epoch 1/30


2022-07-30 07:38:44.498122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


228/228 [==============================] - ETA: 0s - loss: 1.6081 - accuracy: 0.4581

2022-07-30 07:38:54.309405: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


228/228 [==============================] - 11s 46ms/step - loss: 1.6081 - accuracy: 0.4581 - val_loss: 5.4079 - val_accuracy: 0.3084
Epoch 2/30
228/228 [==============================] - 9s 39ms/step - loss: 0.9798 - accuracy: 0.6770 - val_loss: 1.0841 - val_accuracy: 0.6432
Epoch 3/30
228/228 [==============================] - 9s 39ms/step - loss: 0.7772 - accuracy: 0.7407 - val_loss: 0.8226 - val_accuracy: 0.7355
Epoch 4/30
228/228 [==============================] - 9s 40ms/step - loss: 0.6641 - accuracy: 0.7746 - val_loss: 0.6778 - val_accuracy: 0.7839
Epoch 5/30
228/228 [==============================] - 9s 39ms/step - loss: 0.5817 - accuracy: 0.8036 - val_loss: 0.6663 - val_accuracy: 0.7955
Epoch 6/30
228/228 [==============================] - 10s 42ms/step - loss: 0.5174 - accuracy: 0.8296 - val_loss: 0.6162 - val_accuracy: 0.8065
Epoch 7/30
228/228 [==============================] - 9s 41ms/step - loss: 0.4645 - accuracy: 0.8526 - val_loss: 0.6744 - val_accuracy: 0.7942
Epoch 8/

In [155]:
# colors_1_model.evaluate(colors_1_test)

colors_1_model_eval = keras.models.load_model("models/colors_1_3_secondAug_checkpoint_path.keras")
colors_1_model_eval.evaluate(colors_1_test)

2022-07-30 07:41:08.396634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


49/49 [==============================] - 1s 20ms/step - loss: 0.7045 - accuracy: 0.8027


[0.7044712901115417, 0.8026992082595825]

# 2 Vehicle Type classification

### Import train, test, and val datasets

In [158]:
vehicle_type_train = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/train/", image_size=(224, 224), batch_size=16)
vehicle_type_val = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/val", image_size=(224, 224), batch_size=16)
vehicle_type_test = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/test", image_size=(224, 224), batch_size=16)

Found 840 files belonging to 3 classes.
Found 180 files belonging to 3 classes.
Found 180 files belonging to 3 classes.


### Augmentation

In [159]:
vehicle_type_data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

### Define Model

In [160]:
vehicle_type_inputs = keras.Input(shape=(224, 224, 3))

x = vehicle_type_data_augmentation(vehicle_type_inputs)

x = layers.Rescaling(1./255)(vehicle_type_inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation="relu")(x)

# x = layers.Flatten()(x)
x = layers.GlobalAveragePooling2D()(x)

vehicle_type_outputs = layers.Dense(3, activation="softmax")(x)

vehicle_type_model = keras.Model(inputs=vehicle_type_inputs, outputs=vehicle_type_outputs)

vehicle_type_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

vehicle_type_model.summary()

Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_27 (Rescaling)    (None, 224, 224, 3)       0         
                                                                 
 conv2d_119 (Conv2D)         (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_92 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_120 (Conv2D)         (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_93 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                     

### Callbacks

In [162]:
vehicle_type_callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/vehicle_type_fourAug_checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

### Run the Model

In [163]:
history = vehicle_type_model.fit(
  vehicle_type_train.cache(),
  epochs=30,
  callbacks=vehicle_type_callbacks_list,
  validation_data=vehicle_type_val.cache()
)

Epoch 1/30


2022-07-30 07:45:15.223172: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - ETA: 0s - loss: 1.2498 - accuracy: 0.3548

2022-07-30 07:45:22.735823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 9s 144ms/step - loss: 1.2498 - accuracy: 0.3548 - val_loss: 1.0517 - val_accuracy: 0.4667
Epoch 2/30
53/53 [==============================] - 7s 128ms/step - loss: 1.0634 - accuracy: 0.4536 - val_loss: 1.8007 - val_accuracy: 0.3333
Epoch 3/30
53/53 [==============================] - 7s 128ms/step - loss: 0.9977 - accuracy: 0.5512 - val_loss: 0.8311 - val_accuracy: 0.6333
Epoch 4/30
53/53 [==============================] - 7s 124ms/step - loss: 0.9066 - accuracy: 0.6048 - val_loss: 1.0043 - val_accuracy: 0.5500
Epoch 5/30
53/53 [==============================] - 7s 129ms/step - loss: 0.8322 - accuracy: 0.6369 - val_loss: 0.7349 - val_accuracy: 0.6611
Epoch 6/30
53/53 [==============================] - 7s 130ms/step - loss: 0.7556 - accuracy: 0.6774 - val_loss: 0.6798 - val_accuracy: 0.6944
Epoch 7/30
53/53 [==============================] - 7s 131ms/step - loss: 0.6374 - accuracy: 0.7381 - val_loss: 1.2025 - val_accuracy: 0.5167
Epoch 8/30
53/53 

In [164]:
vehicle_types_eval = keras.models.load_model("models/vehicle_type_fourAug_checkpoint_path.keras")
vehicle_types_eval.evaluate(vehicle_type_test)
# vehicle_type_model.evaluate(vehicle_type_test)

2022-07-30 07:48:11.046077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 1s 47ms/step - loss: 0.3121 - accuracy: 0.9056


[0.3120936453342438, 0.9055556058883667]

# 3 Colors_2 dataset

In [165]:
IMAGE_SIZE = 90
BATCH_SIZE = 32

In [166]:
colors_2_train = utils.image_dataset_from_directory("datasets/data/split/colors_2/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_2_val = utils.image_dataset_from_directory("datasets/data/split/colors_2/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_2_test = utils.image_dataset_from_directory("datasets/data/split/colors_2/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 10917 files belonging to 8 classes.
Found 2337 files belonging to 8 classes.
Found 2347 files belonging to 8 classes.


### Augmentation

In [167]:
colors_2_data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

In [168]:
colors_2_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

x = colors_2_data_augmentation(colors_2_inputs)

x = layers.Rescaling(1./255)(colors_2_inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.GlobalAveragePooling2D()(x)

x = layers.Flatten()(x)

colors_2_outputs = layers.Dense(8, activation="softmax")(x)

colors_2_model = keras.Model(inputs=colors_2_inputs, outputs=colors_2_outputs)

colors_2_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

colors_2_model.summary()

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 90, 90, 3)]       0         
                                                                 
 rescaling_28 (Rescaling)    (None, 90, 90, 3)         0         
                                                                 
 conv2d_124 (Conv2D)         (None, 88, 88, 32)        896       
                                                                 
 max_pooling2d_96 (MaxPoolin  (None, 44, 44, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_125 (Conv2D)         (None, 42, 42, 64)        18496     
                                                                 
 max_pooling2d_97 (MaxPoolin  (None, 21, 21, 64)       0         
 g2D)                                                     

### Callbacks

In [169]:
colors_2_callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/colors_2_afterAug_checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

In [170]:
history = colors_2_model.fit(
  colors_2_train.cache(),
  epochs=30,
  callbacks=colors_2_callbacks_list,
  validation_data=colors_2_val.cache()
)

Epoch 1/30


2022-07-30 07:50:20.595652: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


342/342 [==============================] - ETA: 0s - loss: 0.9576 - accuracy: 0.6757

2022-07-30 07:50:35.202758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


342/342 [==============================] - 16s 45ms/step - loss: 0.9576 - accuracy: 0.6757 - val_loss: 0.7221 - val_accuracy: 0.7608
Epoch 2/30
342/342 [==============================] - 13s 39ms/step - loss: 0.4786 - accuracy: 0.8380 - val_loss: 0.4633 - val_accuracy: 0.8434
Epoch 3/30
342/342 [==============================] - 14s 40ms/step - loss: 0.3455 - accuracy: 0.8810 - val_loss: 0.3401 - val_accuracy: 0.8806
Epoch 4/30
342/342 [==============================] - 14s 41ms/step - loss: 0.2770 - accuracy: 0.9047 - val_loss: 0.3644 - val_accuracy: 0.8832
Epoch 5/30
342/342 [==============================] - 15s 44ms/step - loss: 0.2365 - accuracy: 0.9206 - val_loss: 0.4388 - val_accuracy: 0.8789
Epoch 6/30
342/342 [==============================] - 15s 43ms/step - loss: 0.1972 - accuracy: 0.9327 - val_loss: 0.3675 - val_accuracy: 0.8896
Epoch 7/30
342/342 [==============================] - 16s 45ms/step - loss: 0.1807 - accuracy: 0.9413 - val_loss: 0.3477 - val_accuracy: 0.9033
Epo

In [171]:
# colors_2_model.evaluate(colors_2_test)

colors_2_eval = keras.models.load_model("models/colors_2_afterAug_checkpoint_path.keras")
colors_2_eval.evaluate(colors_2_test)


2022-07-30 08:03:26.666136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


74/74 [==============================] - 2s 19ms/step - loss: 0.3916 - accuracy: 0.8624


[0.3916303217411041, 0.8623774647712708]